As seguintes bibliotecas adicionais são necessárias para executar este
notebook. Observe que a execução no Colab é experimental, por favor, relate um Github
questão se você tiver algum problema.


In [ ]:
!pip install d2l==1.0.3


# O conjunto de dados para pré-treinamento BERT
:label:`conjunto de dados sec_bert`

Para pré-treinar o modelo BERT conforme implementado em :numref:`sec_bert`,
precisamos gerar o conjunto de dados no formato ideal para facilitar
as duas tarefas de pré-treinamento:
modelagem de linguagem mascarada e previsão da próxima frase.
Por um lado,
o modelo BERT original é pré-treinado na concatenação de
dois grandes corpora BookCorpus e Wikipedia em inglês (veja :numref:`subsec_bert_pretraining_tasks`),
tornando difícil para a maioria dos leitores deste livro correr.
Por outro lado,
o modelo BERT pré-treinado pronto para uso
pode não ser adequado para aplicações de domínios específicos, como medicina.
Por isso, está se tornando popular pré-treinar o BERT em um conjunto de dados personalizado.
Para facilitar a demonstração do pré-treinamento BERT,
usamos um corpus menor WikiText-2 :cite:`Merity.Xiong.Bradbury.ea.2016`.

Comparando com o conjunto de dados PTB usado para pré-treinamento word2vec em :numref:`sec_word2vec_data`,
O WikiText-2 (i) mantém a pontuação original, tornando-o adequado para a previsão da próxima frase; (ii) mantém o caso e os números originais; (iii) é mais de duas vezes maior.


In [11]:
import os
import random
import torch
from d2l import torch as d2l

No [**conjunto de dados WikiText-2**],
cada linha representa um parágrafo onde
um espaço é inserido entre qualquer pontuação e seu token precedente.
Parágrafos com pelo menos duas frases são mantidos.
Para dividir frases, usamos apenas o ponto como delimitador para simplificar.
Deixamos as discussões sobre técnicas mais complexas de divisão de frases nos exercícios
no final desta seção.


In [21]:
#@save
d2l.DATA_HUB['wikitext-2'] = (
    'http://la.ihainan.me/'
    'wikitext-2-v1.zip', 'f6e734fc17885b364243f67b30385a3d')

#@save
def _read_wiki(data_dir):
    file_name = os.path.join(data_dir, 'wiki.train.tokens')
    with open(file_name, 'r') as f:
        lines = f.readlines()
    # Uppercase letters are converted to lowercase ones
    paragraphs = [line.strip().lower().split(' . ')
                  for line in lines if len(line.split(' . ')) >= 2]
    random.shuffle(paragraphs)
    return paragraphs

## Definindo funções auxiliares para tarefas de pré-treinamento

A seguir,
começamos implementando funções auxiliares para as duas tarefas de pré-treinamento de BERT:
previsão da próxima frase e modelagem de linguagem mascarada.
Essas funções auxiliares serão invocadas mais tarde
ao transformar o corpus de texto bruto
no conjunto de dados do formato ideal para pré-treinar o BERT.

### [**Gerando a próxima tarefa de previsão de frases**]

De acordo com as descrições de :numref:`subsec_nsp`,
a função `_get_next_sentence` gera um exemplo de treinamento
para a tarefa de classificação binária.


In [13]:
#@save
def _get_next_sentence(sentence, next_sentence, paragraphs):
    if random.random() < 0.5:
        is_next = True
    else:
        # `paragraphs` is a list of lists of lists
        next_sentence = random.choice(random.choice(paragraphs))
        is_next = False
    return sentence, next_sentence, is_next

A função a seguir gera exemplos de treinamento para a previsão da próxima frase
da entrada `paragraph` invocando a função `_get_next_sentence`.
Aqui `parágrafo` é uma lista de frases, onde cada frase é uma lista de tokens.
O argumento `max_len` especifica o comprimento máximo de uma sequência de entrada BERT durante o pré-treinamento.


In [14]:
#@save
def _get_nsp_data_from_paragraph(paragraph, paragraphs, vocab, max_len):
    nsp_data_from_paragraph = []
    for i in range(len(paragraph) - 1):
        tokens_a, tokens_b, is_next = _get_next_sentence(
            paragraph[i], paragraph[i + 1], paragraphs)
        # Consider 1 '<cls>' token and 2 '<sep>' tokens
        if len(tokens_a) + len(tokens_b) + 3 > max_len:
            continue
        tokens, segments = d2l.get_tokens_and_segments(tokens_a, tokens_b)
        nsp_data_from_paragraph.append((tokens, segments, is_next))
    return nsp_data_from_paragraph

### [**Gerando a Tarefa de Modelagem de Linguagem Mascarada**]
:label:`subsec_prepare_mlm_data`

Para gerar exemplos de treinamento
para a tarefa de modelagem de linguagem mascarada
de uma sequência de entrada BERT,
definimos a seguinte função `_replace_mlm_tokens`.
Em suas entradas, `tokens` é uma lista de tokens que representam uma sequência de entrada BERT,
`candidate_pred_positions` é uma lista de índices de token da sequência de entrada BERT
excluindo aqueles de tokens especiais (tokens especiais não são previstos na tarefa de modelagem de linguagem mascarada),
e `num_mlm_preds` indica o número de previsões (lembre-se de 15% de tokens aleatórios para prever).
Seguindo a definição da tarefa de modelagem de linguagem mascarada em :numref:`subsec_mlm`,
em cada posição de previsão, a entrada pode ser substituída por
um token especial “&lt;mask&gt;” ou um token aleatório, ou permanecer inalterado.
No final, a função retorna os tokens de entrada após possível substituição,
os índices de token onde as previsões ocorrem e rótulos para essas previsões.


In [15]:
#@save
def _replace_mlm_tokens(tokens, candidate_pred_positions, num_mlm_preds,
                        vocab):
    # For the input of a masked language model, make a new copy of tokens and
    # replace some of them by '<mask>' or random tokens
    mlm_input_tokens = [token for token in tokens]
    pred_positions_and_labels = []
    # Shuffle for getting 15% random tokens for prediction in the masked
    # language modeling task
    random.shuffle(candidate_pred_positions)
    for mlm_pred_position in candidate_pred_positions:
        if len(pred_positions_and_labels) >= num_mlm_preds:
            break
        masked_token = None
        # 80% of the time: replace the word with the '<mask>' token
        if random.random() < 0.8:
            masked_token = '<mask>'
        else:
            # 10% of the time: keep the word unchanged
            if random.random() < 0.5:
                masked_token = tokens[mlm_pred_position]
            # 10% of the time: replace the word with a random word
            else:
                masked_token = random.choice(vocab.idx_to_token)
        mlm_input_tokens[mlm_pred_position] = masked_token
        pred_positions_and_labels.append(
            (mlm_pred_position, tokens[mlm_pred_position]))
    return mlm_input_tokens, pred_positions_and_labels

Ao invocar a função `_replace_mlm_tokens` mencionada anteriormente,
a função a seguir recebe uma sequência de entrada BERT (`tokens`)
como uma entrada e retorna índices dos tokens de entrada
(após possível substituição de token conforme descrito em :numref:`subsec_mlm`),
os índices de token onde as previsões ocorrem,
e índices de rótulos para essas previsões.


In [16]:
#@save
def _get_mlm_data_from_tokens(tokens, vocab):
    candidate_pred_positions = []
    # `tokens` is a list of strings
    for i, token in enumerate(tokens):
        # Special tokens are not predicted in the masked language modeling
        # task
        if token in ['<cls>', '<sep>']:
            continue
        candidate_pred_positions.append(i)
    # 15% of random tokens are predicted in the masked language modeling task
    num_mlm_preds = max(1, round(len(tokens) * 0.15))
    mlm_input_tokens, pred_positions_and_labels = _replace_mlm_tokens(
        tokens, candidate_pred_positions, num_mlm_preds, vocab)
    pred_positions_and_labels = sorted(pred_positions_and_labels,
                                       key=lambda x: x[0])
    pred_positions = [v[0] for v in pred_positions_and_labels]
    mlm_pred_labels = [v[1] for v in pred_positions_and_labels]
    return vocab[mlm_input_tokens], pred_positions, vocab[mlm_pred_labels]

## Transformando texto no conjunto de dados de pré-treinamento

Agora estamos quase prontos para personalizar uma classe `Dataset` para pré-treinamento de BERT.
Antes disso,
ainda precisamos definir uma função auxiliar `_pad_bert_inputs`
para [**acrescentar os tokens especiais “&lt;pad&gt;” às entradas.**]
Seu argumento `examples` contém as saídas das funções auxiliares `_get_nsp_data_from_paragraph` e `_get_mlm_data_from_tokens` para as duas tarefas de pré-treinamento.


In [17]:
#@save
def _pad_bert_inputs(examples, max_len, vocab):
    max_num_mlm_preds = round(max_len * 0.15)
    all_token_ids, all_segments, valid_lens,  = [], [], []
    all_pred_positions, all_mlm_weights, all_mlm_labels = [], [], []
    nsp_labels = []
    for (token_ids, pred_positions, mlm_pred_label_ids, segments,
         is_next) in examples:
        all_token_ids.append(torch.tensor(token_ids + [vocab['<pad>']] * (
            max_len - len(token_ids)), dtype=torch.long))
        all_segments.append(torch.tensor(segments + [0] * (
            max_len - len(segments)), dtype=torch.long))
        # `valid_lens` excludes count of '<pad>' tokens
        valid_lens.append(torch.tensor(len(token_ids), dtype=torch.float32))
        all_pred_positions.append(torch.tensor(pred_positions + [0] * (
            max_num_mlm_preds - len(pred_positions)), dtype=torch.long))
        # Predictions of padded tokens will be filtered out in the loss via
        # multiplication of 0 weights
        all_mlm_weights.append(
            torch.tensor([1.0] * len(mlm_pred_label_ids) + [0.0] * (
                max_num_mlm_preds - len(pred_positions)),
                dtype=torch.float32))
        all_mlm_labels.append(torch.tensor(mlm_pred_label_ids + [0] * (
            max_num_mlm_preds - len(mlm_pred_label_ids)), dtype=torch.long))
        nsp_labels.append(torch.tensor(is_next, dtype=torch.long))
    return (all_token_ids, all_segments, valid_lens, all_pred_positions,
            all_mlm_weights, all_mlm_labels, nsp_labels)

Colocando as funções auxiliares para
gerando exemplos de treinamento das duas tarefas de pré-treinamento,
e a função auxiliar para preencher entradas juntas,
personalizamos a seguinte classe `_WikiTextDataset` como [**o conjunto de dados WikiText-2 para pré-treinamento de BERT**].
Ao implementar a função `__getitem__ `,
podemos acessar arbitrariamente os exemplos de pré-treinamento (modelagem de linguagem mascarada e previsão da próxima frase)
gerado a partir de um par de frases do corpus WikiText-2.

O modelo BERT original usa embeddings WordPiece cujo tamanho de vocabulário é 30000 :cite:`Wu.Schuster.Chen.ea.2016`.
O método de tokenização do WordPiece é uma ligeira modificação do
o algoritmo de codificação de pares de bytes original em :numref:`subsec_Byte_Pair_Encoding`.
Para simplificar, usamos a função `d2l.tokenize` para tokenização.
Tokens pouco frequentes que aparecem menos de cinco vezes são filtrados.


In [18]:
#@save
class _WikiTextDataset(torch.utils.data.Dataset):
    def __init__(self, paragraphs, max_len):
        # Input `paragraphs[i]` is a list of sentence strings representing a
        # paragraph; while output `paragraphs[i]` is a list of sentences
        # representing a paragraph, where each sentence is a list of tokens
        paragraphs = [d2l.tokenize(
            paragraph, token='word') for paragraph in paragraphs]
        sentences = [sentence for paragraph in paragraphs
                     for sentence in paragraph]
        self.vocab = d2l.Vocab(sentences, min_freq=5, reserved_tokens=[
            '<pad>', '<mask>', '<cls>', '<sep>'])
        # Get data for the next sentence prediction task
        examples = []
        for paragraph in paragraphs:
            examples.extend(_get_nsp_data_from_paragraph(
                paragraph, paragraphs, self.vocab, max_len))
        # Get data for the masked language model task
        examples = [(_get_mlm_data_from_tokens(tokens, self.vocab)
                      + (segments, is_next))
                     for tokens, segments, is_next in examples]
        # Pad inputs
        (self.all_token_ids, self.all_segments, self.valid_lens,
         self.all_pred_positions, self.all_mlm_weights,
         self.all_mlm_labels, self.nsp_labels) = _pad_bert_inputs(
            examples, max_len, self.vocab)

    def __getitem__(self, idx):
        return (self.all_token_ids[idx], self.all_segments[idx],
                self.valid_lens[idx], self.all_pred_positions[idx],
                self.all_mlm_weights[idx], self.all_mlm_labels[idx],
                self.nsp_labels[idx])

    def __len__(self):
        return len(self.all_token_ids)

Ao usar a função `_read_wiki` e a classe `_WikiTextDataset`,
definimos o seguinte `load_data_wiki` para [**baixar e conjunto de dados WikiText-2
e gerar exemplos de pré-treinamento**] a partir dele.


In [19]:
#@save
def load_data_wiki(batch_size, max_len):
    """Load the WikiText-2 dataset."""
    num_workers = d2l.get_dataloader_workers()
    data_dir = d2l.download_extract('wikitext-2', 'wikitext-2')
    paragraphs = _read_wiki(data_dir)
    train_set = _WikiTextDataset(paragraphs, max_len)
    train_iter = torch.utils.data.DataLoader(train_set, batch_size,
                                        shuffle=True, num_workers=num_workers)
    return train_iter, train_set.vocab

Definir o tamanho do lote como 512 e o comprimento máximo de uma sequência de entrada BERT como 64,
nós [**imprimimos as formas de um minilote de exemplos de pré-treinamento BERT**].
Observe que em cada sequência de entrada BERT,
$10$ ($64 \vezes 0,15$) posições são previstas para a tarefa de modelagem de linguagem mascarada.


In [22]:
batch_size, max_len = 512, 64
train_iter, vocab = load_data_wiki(batch_size, max_len)

for (tokens_X, segments_X, valid_lens_x, pred_positions_X, mlm_weights_X,
     mlm_Y, nsp_y) in train_iter:
    print(tokens_X.shape, segments_X.shape, valid_lens_x.shape,
          pred_positions_X.shape, mlm_weights_X.shape, mlm_Y.shape,
          nsp_y.shape)
    break

/home/ellizeu.sena/Documentos/Aprendizado Profundo/Processamento de Linguagem Natural/venv/lib/python3.9/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


torch.Size([512, 64]) torch.Size([512, 64]) torch.Size([512]) torch.Size([512, 10]) torch.Size([512, 10]) torch.Size([512, 10]) torch.Size([512])


torch.Size([512, 64]) torch.Size([512, 64]) torch.Size([512]) torch.Size([512, 10]) torch.Size([512, 10]) torch.Size([512, 10]) torch.Size([512])


No final, vamos dar uma olhada no tamanho do vocabulário.
Mesmo depois de filtrar tokens pouco frequentes,
ainda é mais de duas vezes maior que o conjunto de dados PTB.


In [23]:
len(vocab)

20256

## Resumo

* Comparado com o conjunto de dados PTB, o conjunto de dados WikiText-2 mantém a pontuação, caixa e números originais e é mais de duas vezes maior.
* Podemos acessar arbitrariamente os exemplos de pré-treinamento (modelagem de linguagem mascarada e previsão da próxima frase) gerados a partir de um par de frases do corpus WikiText-2.


## Exercícios

1. Para simplificar, o ponto é usado como o único delimitador para dividir sentenças. Tente outras técnicas de divisão de sentenças, como spaCy e NLTK. Tome o NLTK como exemplo. Você precisa instalar o NLTK primeiro: `pip install nltk`. No código, primeiro `import nltk`. Em seguida, baixe o tokenizador de sentenças Punkt: `nltk.download('punkt')`. Para dividir sentenças como `sentences = 'This is great ! Why not ?'`, invocar `nltk.tokenize.sent_tokenize(sentences)` retornará uma lista de duas strings de sentenças: `['This is great !', 'Why not ?']`.
1. Qual é o tamanho do vocabulário se não filtrarmos nenhum token infrequente?


[Discussões](https://discuss.d2l.ai/t/1496)
